In [35]:
# Установка необходимых модулей
!pip install skorch

# Импорт модулей
import numpy as np
import pandas as pd
import os
import zipfile
import torch
import random
import shutil 
import torchvision
import matplotlib.pyplot as plt
import time
import copy
from tqdm import tqdm
from torchvision import transforms, models
from skorch.callbacks import LRScheduler
from skorch import NeuralNetClassifier
from skorch.helper import predefined_split
from skorch.helper import SliceDataset
from sklearn.model_selection import GridSearchCV

In [36]:
# Зафиксируем ГСЧ
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [37]:
# Выполним разархивацию файла
with zipfile.ZipFile('../input/plates.zip', 'r') as zip_obj:
    zip_obj.extractall('/kaggle/working/')
    
print('After zip extraction:')
print(os.listdir("/kaggle/working/"))

After zip extraction:
['plates', '__notebook_source__.ipynb', '__MACOSX', '.ipynb_checkpoints', 'submission.csv']


In [38]:
# Выполним проверку правильности разархивации
data_root = '/kaggle/working/plates/'
print(os.listdir(data_root))

['.DS_Store', 'train', 'test']


In [39]:
# Создадим папку для тренировочного набора данных
train_dir = 'train'
class_names = ['cleaned', 'dirty']

for class_name in class_names:
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)

# Поместим изображения в папку
for class_name in class_names:
    source_dir = os.path.join(data_root, 'train', class_name)
    for i, file_name in enumerate(tqdm(os.listdir(source_dir))):
        dest_dir = os.path.join(train_dir, class_name) 
        shutil.copy(os.path.join(source_dir, file_name), 
                    os.path.join(dest_dir, file_name))

100%|██████████| 21/21 [00:00<00:00, 3926.03it/s]


In [40]:
# Зададим преобразования, которые будут выполнены над изображениями
train_transforms = transforms.Compose([
    transforms.RandomApply([
        transforms.ColorJitter(
            brightness=0.5,
            contrast=0.5,
            saturation=0.5,
            hue=0.5
        )
    ]),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Создадим датасет с изображениями
train_dataset = torch.utils.data.ConcatDataset([
    torchvision.datasets.ImageFolder(train_dir, train_transforms),
    torchvision.datasets.ImageFolder(train_dir, train_transforms),
    torchvision.datasets.ImageFolder(train_dir, train_transforms),
    torchvision.datasets.ImageFolder(train_dir, train_transforms),
    torchvision.datasets.ImageFolder(train_dir, train_transforms),
    torchvision.datasets.ImageFolder(train_dir, train_transforms),
    torchvision.datasets.ImageFolder(train_dir, train_transforms),
    torchvision.datasets.ImageFolder(train_dir, train_transforms)
])

batch_size = 8
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, 
    shuffle=True, num_workers=batch_size)

In [ ]:
    # Класс полносвязной сети для выходного слоя 
    class ModelNet(torch.nn.Module):
        def __init__(self, n_neurons):
            super(ModelNet, self).__init__()
            self.fc1 = torch.nn.Linear(n_neurons, 128)
            self.act1 =  torch.nn.ReLU()
            self.fc2 = torch.nn.Linear(128, 128)
            self.act2 = torch.nn.ReLU()
            self.fc3 = torch.nn.Linear(128, 2)

        def forward(self, x):
            x = self.fc1(x)
            x = self.act1(x)
            x = self.fc2(x)
            x = self.act2(x)
            x = self.fc3(x)
            return x

In [41]:
# Функция, возвращающая модель нейронной сети
def set_model():
    '''
    This function creates model: resnet152 + 3 fc-layers
    '''
    model = models.resnet152(pretrained=True)
    # Включим расчет градиента для всех уровней
    for param in model.parameters():
        param.requires_grad = True
    model.fc = ModelNet(model.fc.in_features)
    return model

model = set_model()

In [42]:
# Реализуем уменьшение шага градиентного спуска
lrscheduler = LRScheduler(policy='StepLR', step_size=3, gamma=0.6)

In [11]:
# Зададим начальные параметры модели
net = NeuralNetClassifier(
    model,
    criterion=torch.nn.CrossEntropyLoss,
    batch_size=batch_size,
    train_split=None,
    device='cuda',
    iterator_train__shuffle=True,
    iterator_train__num_workers=8,
    callbacks=[lrscheduler],
)

In [12]:
# Приведем данные к удобному для работы формату
X_sl = SliceDataset(train_dataset)
y_sl = SliceDataset(train_dataset, idx=1)

In [13]:
# Зададим возможные значения параметров сети, выполним обучение
params = {'optimizer': [torch.optim.Adam],
          'lr': [1.0e-5, 1.0e-4, 1.0e-3],
          'optimizer__weight_decay': [1.0e-3],
          'max_epochs': [15, 30]}
gs = GridSearchCV(net, params, cv=5, scoring='accuracy')
# Выполним поиск наилучших параметров сети
gs.fit(X_sl, y_sl)

  epoch    train_loss      dur
-------  ------------  -------
      1        0.6800  10.5988
      2        0.6404  9.1801
      3        0.5743  9.3180
      4        0.4838  9.0470
      5        0.3755  9.0978
      6        0.3023  9.2663
      7        0.2703  10.3296
      8        0.2514  9.3957
      9        0.2513  9.1429
     10        0.2257  9.0607
     11        0.1837  9.0313
     12        0.2029  8.8795
     13        0.1673  9.4864
     14        0.1599  10.0441
     15        0.1770  9.2353
  epoch    train_loss     dur
-------  ------------  ------
      1        0.6813  9.3344
      2        0.6417  9.2496
      3        0.5702  9.2981
      4        0.4550  9.5212
      5        0.3717  9.4573
      6        0.3297  9.3894
      7        0.2593  9.1298
      8        0.2422  9.2067
      9        0.2266  9.0818
     10        0.2357  9.5389
     11        0.2053  9.4636
     12        0.1953  9.3363
     13        0.2201  9.1346
     14        0.1718  9.1201
     

     21        0.1479  9.7548
     22        0.1458  9.1523
     23        0.1546  9.1168
     24        0.1628  9.4175
     25        0.1621  9.3487
     26        0.1375  9.1127
     27        0.1476  9.5716
     28        0.1476  9.7944
     29        0.1850  8.8893
     30        0.1496  9.0548
  epoch    train_loss     dur
-------  ------------  ------
      1        0.5075  9.4320
      2        0.2427  9.3408
      3        0.3709  9.6212
      4        0.3280  9.1464
      5        0.2320  9.1850
      6        0.1159  8.9512
      7        0.1846  9.3026
      8        0.1171  9.2537
      9        0.2206  9.7712
     10        0.1057  9.0191
     11        0.1115  9.0168
     12        0.1236  9.0551
     13        0.0669  9.1375
     14        0.0618  9.0518
     15        0.0994  9.1722
  epoch    train_loss     dur
-------  ------------  ------
      1        0.5402  8.9716
      2        0.2758  9.2387
      3        0.2599  9.3138
      4        0.2296  9.3399
      5   

     19        0.1074  9.2765
     20        0.0595  9.3139
     21        0.0364  8.9909
     22        0.0387  9.0304
     23        0.0679  9.2051
     24        0.0518  9.7237
     25        0.0437  9.4032
     26        0.0696  9.3497
     27        0.0385  9.3618
     28        0.1020  9.1168
     29        0.0421  8.9835
     30        0.0481  9.3444
  epoch    train_loss     dur
-------  ------------  ------
      1        0.7012  9.4690
      2        0.7039  9.3050
      3        0.6984  9.3536
      4        0.6994  9.0848
      5        0.6976  9.0150
      6        0.6938  10.3485
      7        0.6940  8.8905
      8        0.6883  9.1922
      9        0.6967  9.2261
     10        0.6934  9.3291
     11        0.6920  8.9410
     12        0.6901  9.3289
     13        0.6893  9.7017
     14        0.6857  9.1256
     15        0.6778  9.2129
  epoch    train_loss     dur
-------  ------------  ------
      1        0.6985  9.3370
      2        0.7085  9.3410
      3  

     20        0.6852  8.9951
     21        0.6880  9.7198
     22        0.6940  9.3886
     23        0.6905  8.8739
     24        0.6907  9.2312
     25        0.6799  9.1437
     26        0.6909  9.3119
     27        0.6860  9.1472
     28        0.6818  9.6215
     29        0.6837  9.0803
     30        0.6799  9.2213
  epoch    train_loss      dur
-------  ------------  -------
      1        0.4987  11.0838
      2        0.3135  11.0287
      3        0.2957  11.6704
      4        0.1945  11.0660
      5        0.1760  11.1328
      6        0.2000  11.1845
      7        0.1706  11.1992
      8        0.1169  12.2519
      9        0.0726  10.8894
     10        0.1062  10.7788
     11        0.0754  10.8158
     12        0.0588  10.9001
     13        0.0711  11.7388
     14        0.1068  11.1669
     15        0.1150  10.8106
     16        0.0320  10.8436
     17        0.0777  10.6942
     18        0.0992  11.2493
     19        0.0769  11.5830
     20        0.07

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=<class 'skorch.classifier.NeuralNetClassifier'>[uninitialized](
  module=ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, str...
      (fc2): Linear(in_features=128, out_features=128, bias=True)
      (act2): ReLU()
      (fc3): Linear(in_features=128, out_features=2, bias=True)
    )
  ),
),
             iid='warn', n_jobs=None,
             param_grid={'lr': [1e-05, 0.0001, 0.001], 'max_epochs': [15, 30],
                         'optimizer': [<class 'torch.optim.adam.Adam'>],
                         'optimizer__weight_decay': [0.001]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [14]:
# Выведем наилучшие параметры сети
result = gs.best_params_
gs.best_params_

{'lr': 0.0001,
 'max_epochs': 30,
 'optimizer': torch.optim.adam.Adam,
 'optimizer__weight_decay': 0.001}

In [15]:
gs.cv_results_

{'mean_fit_time': array([146.00534959, 285.59886847, 145.19197793, 283.95779476,
        145.10002098, 283.62053204]),
 'std_fit_time': array([0.9976515 , 0.42581014, 0.48622393, 1.25186575, 0.39448992,
        0.36345732]),
 'mean_score_time': array([3.95416985, 4.11363239, 4.15013285, 4.03192296, 4.02667861,
        3.97904644]),
 'std_score_time': array([0.10203793, 0.15657464, 0.21766283, 0.15836999, 0.08297086,
        0.12829872]),
 'param_lr': masked_array(data=[1e-05, 1e-05, 0.0001, 0.0001, 0.001, 0.001],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_max_epochs': masked_array(data=[15, 30, 15, 30, 15, 30],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_optimizer': masked_array(data=[<class 'torch.optim.adam.Adam'>,
                    <class 'torch.optim.adam.Adam'>,
                    <class 'torch.optim.adam.Adam'>,
        

In [16]:
# Выполним копирование изображений в папку unknown
# это необходимо, чтобы задать метку класса
test_dir = 'test'
shutil.copytree(os.path.join(data_root, 'test'), 
                os.path.join(test_dir, 'unknown'))

'test/unknown'

In [17]:
# Дополним специальный метод __getitem__ возвратом пути к изображению
class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        path = self.imgs[index][0]
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path
    
test_dataset = ImageFolderWithPaths('/kaggle/working/test', test_transforms)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False)

In [18]:
# Выполним классификацию тестового набора данных
test_predictions = []
test_img_paths = []

model = gs.best_estimator_
sm = torch.nn.Softmax(dim=1)

for inputs, labels, paths in tqdm(test_dataloader):
    inputs = inputs
    labels = labels
    preds = model.predict_proba(inputs)
    test_predictions.extend(preds)
    test_img_paths.extend(paths)
test_predictions = torch.tensor(test_predictions)
test_predictions = sm(test_predictions)
test_predictions = test_predictions[:, 1].numpy()

100%|██████████| 93/93 [00:08<00:00, 10.56it/s]


In [43]:
# Создадим и заполним датафрейм с результатами
submission_df = pd.DataFrame.from_dict({'id': test_img_paths, 'label': test_predictions})
submission_df['label'] = submission_df['label'].map(lambda pred: 'dirty' if pred > 0.7 else 'cleaned')
submission_df['id'] = submission_df['id'].str.replace('/kaggle/working/test/unknown/', '')
submission_df['id'] = submission_df['id'].str.replace('.jpg', '')
submission_df.set_index('id', inplace=True)
submission_df.head(n=6)

,label
id,
0000,dirty
0001,dirty
0002,dirty
0003,dirty
0004,cleaned
0005,dirty


In [34]:
# Преобразуем датафрейм в csv файл
submission_df.to_csv('submission.csv')

In [32]:
# Очистим файловое пространство от ненужных данных
!rm -rf train test